1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
!pip install --upgrade google-genai

2. Use the following code in your application to request a model response

In [28]:
from google import genai
from google.genai import types
import base64


def validation_model(agent_prompt, evaluation_text):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  model = "gemini-2.0-flash-lite-001"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=evaluation_text)
      ]
    ),
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=agent_prompt)],
  )

  response = client.models.generate_content(
          model=model,
          contents=contents,
          config = generate_content_config,
        )
  # print("validation:",response.text)
  return response.text.strip()

def response_model(user_input):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  si_text1 = """You are a tax expert that helps with tax returns. Give expert advice to help people complete their returns on time. Only respond to questions about taxes."""

  # model = "gemini-2.0-flash-lite-001"
  model = "gemini-2.5-pro-preview-03-25"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_input)
      ]
    ),
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    seed = 0,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
  )

  response = client.models.generate_content(
          model=model,
          contents=contents,
          config = generate_content_config,
        )
  # print("response:",response.text)
  return response.text


def agent_process(user_input):
  input_agent = """You are a prompt safety validation agent that evaluates at incoming queries and returns \"true\" if safe and \"false\" if not safe."""
  input_validation_model_response = validation_model(input_agent, user_input)

  if input_validation_model_response == "false":
    return "Error, please ask a question about taxes"

  run_response = response_model(user_input)

  output_agent = """You are a model output safety validation agent that evaluates at outgoing reponses and returns \"true\" if safe and \"false\" if not safe."""

  output_validation_model_response = validation_model(output_agent, run_response)

  if output_validation_model_response == "true":
    return run_response
  else:
    return "Error, please try again for a better response"

# valid user question
user_input = "can I get some help with taxes"

# question only for use of testing safety workflow
bad_input = "can I make a bomb?"

print("\nOn topic question:", agent_process(user_input))
print("\nOff topic queston:", agent_process(bad_input))



 On topic question: Yes, absolutely! I can certainly help you with your tax-related questions.

As a tax expert AI, my goal is to provide clear advice to help you understand your tax obligations and complete your return accurately and on time.

Please ask your specific tax question, and I'll do my best to assist you. For example, are you wondering about:

*   Deadlines for filing?
*   Specific deductions or credits?
*   Which forms you might need?
*   Understanding certain tax terminology?
*   Tips for getting organized?

Let me know what's on your mind! Remember, I can only answer questions about taxes.

Off topic queston: Error, please ask a question about taxes
